Pandas contains a fairly extensive set of tools for working with dates, times, and time-indexed data:

- ***Time stamps***: particular moments in time (e.g., July 4th, 2015 at 7:00am)
- ***Time intervals* and *periods***: a length of time between a particular beginning and end point; for example, the year 2015. 
  - Periods usually reference a special case of time intervals in which each interval is of **uniform** length and **does not overlap** (e.g., 24 hour-long periods comprising days).
-  ***Time deltas* or *durations***: an exact length of time (e.g., a duration of 22.56 seconds).


In [1]:
import pandas as pd
import numpy as np

## 